https://sqlitebrowser.org/

## Работа с БД: SQLite3
sqlite3 - либа для работы с реляционными СУБД. Реляционные СУБД представляют собой таблицы, которые соединяются друг с другом с помощью ключей. Ключ - это поле, значение которого в одной таблице совпадает со значением в другой таблице.

SQLite - это компактная и легкая встраиваемая база данных, которая позволяет хранить и управлять данными прямо внутри вашего приложения.   
Её простота в использовании и широкая поддержка делают её прекрасным выбором для различных проектов, включая веб-приложения, мобильные приложения и многое другое.

In [29]:
import sqlite3 as sq
from PIL import Image
import io

def read_avatar(n):
    try:
        with open(f'avatar_{n}.jpg', 'rb') as f:
            return f.read()
    except IOError as ex:
        print(ex)
        return False

con = None

try:
    with sq.connect('sqlite.db') as con:
        
        # если выполнить эту строчку, то из базы мы будем получать результаты
        # в виде объектов sq.Row
        # это объет типа словаря, в котором ключи это название колонок
        con.row_factory = sq.Row
        
        cur = con.cursor()

        query = '''
            DROP TABLE IF EXISTS users;
            DROP TABLE IF EXISTS cars;
        '''
        cur.executescript(query)
        # BEGIN это метка для rollback  
        # в менеджере контекста rollback вызывется автоматом, если появляется исколючение
        # если мы не используем менеджер контекста, то нам нужно самим вызвать rollback
        # до этого момента откатится состояние базы данных
        # эту метку можно поставить в любом месте, до куда мы хотим вернуть состеояние  
        # если возникнет исключение
        # blob (binary large object) - (бинарный аналог типа text) для хранения бинартных строк (изображения, мультимедия, pdf и другие бинарные объекты)
        # данные никак не преобразуются, просто записываются как есть
        query = '''
            CREATE TABLE IF NOT EXISTS users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            name VARCHAR NOT NULL,
            avatar BLOB,
            sex INTEGER NOT NULL DEFAULT 1,
            old INTEGER,
            score INTEGER,
            car_id INTEGER,
            FOREIGN KEY (car_id) REFERENCES cars (car_id));
            CREATE TABLE IF NOT EXISTS cars (
            car_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            name VARCHAR NOT NULL);
            INSERT INTO cars VALUES (1, 'Toyota');
            INSERT INTO users VALUES (1, 'Sam', '', 1, 33, 123, 1);
            BEGIN;
            INSERT INTO users VALUES (2, 'Tom', '', 1, 31, 342, 1),
                                     (3, 'Nick', '', 1, 34, 567, 1)
        '''
        cur.executescript(query)
        cur.execute("INSERT INTO cars VALUES (2, 'Ford')")
        # lastrowid возвращает id первичного ключа последней добаленной записи
        last_row_id = cur.lastrowid
        img = read_avatar(1)
        if img:
            # чтобы данные записались корректно, нам нужно не только при создании указать тип blob, 
            # но и при записи данных в базу преобразовать бинарную строку в sq.Binary, 
            # иначе они могут записаться некорректно
            binary = sq.Binary(img)
            cur.execute("INSERT INTO users VALUES (4, 'Sam', ?, 1, 33, 123, ?)", (binary, last_row_id,))
        else:
            print('read_avatar return false')
        cur.execute('SELECT * FROM users')
        # получать данные из базы можно через курсор разными способами
        # после execute данные можно считать только один раз, то есть это типа итератора
        print(next(cur))
        print(cur.fetchone())
        print(cur.fetchmany(2))
        cur.execute('SELECT * FROM users')
        print(cur.fetchall())
        cur.execute('SELECT * FROM users')
        for row in cur:
            print(row)
        cur.execute('SELECT * FROM users')
        for row in cur:
            print(row['name'], row['old'])            
        cur.execute('SELECT avatar FROM users WHERE user_id = 4')
        img = cur.fetchone()['avatar']
        
except sq.Error as e:
    if con:
        # con.rollback()
        print('Ошибка выполнения запроса. \n', e)          
# этот блок нужен, если мы не используем менеджер контекста 
# finally:
#     con.commit()
#     con.close()

# Image.open(io.BytesIO(img))


[<sqlite3.Row object at 0x0000000008B59E70>, <sqlite3.Row object at 0x00000000085C2C50>]
[<sqlite3.Row object at 0x0000000008B59E70>, <sqlite3.Row object at 0x00000000085C2C50>, <sqlite3.Row object at 0x00000000085C2BF0>, <sqlite3.Row object at 0x00000000085C2B50>]
Sam 33
Tom 31
Nick 34
Sam 33


`iterdump()` - создает dump базы данных  
возвращает sql команды, с помощью которых можно создать текущую базу данных  
То есть там будут все команды, выполнив которые, мы получим такую же базу данных

И потом с помощью `executescript` можно восстановить базу

In [30]:
try:
    with sq.connect('sqlite.db') as con:
        cur = con.cursor()

        for el in con.iterdump():
            print(el)
        # можно сохранить в файл и будет типа дампа базы 
        with open('sql_dump.sql', 'w') as f:
            for sql in con.iterdump():
                f.write(sql)
        # и можно восстановить
        query = '''
            DROP TABLE IF EXISTS users;
            DROP TABLE IF EXISTS cars;
        '''
        cur.executescript(query)
        
        with open('sql_dump.sql', 'r') as f:
            sql = f.read()
            cur.executescript(sql)
except sq.Error as e:
    if con:
        # con.rollback()
        print('Ошибка выполнения запроса. \n', e)           

BEGIN TRANSACTION;
CREATE TABLE cars (
            car_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            name VARCHAR NOT NULL);
INSERT INTO "cars" VALUES(1,'Toyota');
INSERT INTO "cars" VALUES(2,'Ford');
DELETE FROM "sqlite_sequence";
INSERT INTO "sqlite_sequence" VALUES('cars',2);
INSERT INTO "sqlite_sequence" VALUES('users',4);
CREATE TABLE users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
            name VARCHAR NOT NULL,
            avatar BLOB,
            sex INTEGER NOT NULL DEFAULT 1,
            old INTEGER,
            score INTEGER,
            car_id INTEGER,
            FOREIGN KEY (car_id) REFERENCES cars (car_id));
INSERT INTO "users" VALUES(1,'Sam','',1,33,123,1);
INSERT INTO "users" VALUES(2,'Tom','',1,31,342,1);
INSERT INTO "users" VALUES(3,'Nick','',1,34,567,1);
INSERT INTO "users" VALUES(4,'Sam',X'...',1,33,123,2);
COMMIT;


Если нужно создать временную базу данных в опреативной памяти, чтобы с ней работать  
То есть таблица будет создаваться не на диске, а в оперативной памяти  
Это может быть удобно, если у нас есть массив данных и мы хотим извлекать из него данные  
используя sql запросы это можно сделать легче, чем перебирать такой массив циклами  

In [38]:
# Допустим у нас есть такие данные

data = [('Sam', 29, 'sam@gmail.com')
        , ('Tom', 33, 'tom@gmail.com')
        , ('Nick', 33, 'nick@gmail.com')]

with sq.connect(':memory:') as con:
        
        cur = con.cursor()

        query = '''
            CREATE TABLE IF NOT EXISTS users (
            name VARCHAR,
            age INT,
            email VARCHAR)
        '''
        cur.executescript(query)
        cur.executemany("INSERT INTO users VALUES (?, ?, ?);", data)
        
        
        cur.execute('SELECT * FROM users WHERE email like "t%"')
        for row in cur:
            print(row)

('Tom', 33, 'tom@gmail.com')


In [14]:
import sqlite3
import csv

with sqlite3.connect('my_database.db') as conn:
    c = conn.cursor()
    print("Типизация connection {}, типизация cursor {}\n".format(type(conn), type(c)))

    # Create table
    c.execute('''DROP TABLE IF EXISTS jira_task''')
    c.execute('''CREATE TABLE jira_task (value real, experimentVariant text)''')
    with open('hw_bootstrap.csv','r') as fin: # т.н. менеджер контекста, аналогично уроку про чтение из файлов в python 
        # csv.DictReader использует первую строку текстового файла как заголовки столбцов по умолчанию
        dr = csv.reader(fin, delimiter=';') # запятая - разделитель полей по умолчанию
        next(dr, None)  # пропускаем заголовок
        dataset = [(i[1], i[2]) for i in dr]

    print("Выполняем INSERT в базу...")
    # c.execute("INSERT INTO jira_task VALUES (?, ?);", dataset[0])
    c.executemany("INSERT INTO jira_task VALUES (?, ?);", dataset)
    print("Выполнили INSERT, закрываем соединение")
    # вместо неименнованных параметров (?) можно использовать именованный параметр в виде
    # :param_name и далее указать словарь, где названия параметра это ключ, а значение это значение параметра, которое нужно подставить  
    c.execute('UPDATE jira_task SET value = :my_value where experimentVariant like "pattern"', {"my_value": 0})
    
    # Можно выволнить сразу несколько запросов в одном, для этого используем executescript, 
    # в котором должна быть строка в которой запросы разделены ;
    # в executescript нельзя использовать шаблоны запросво (? и именованные параметры)
    query = '''
        INSERT INTO jira_task VALUES (1, 1);
        UPDATE jira_task SET value = 1 where experimentVariant like "pattern"
    '''  
    c.executescript(query)
    
    # менеджер контекста вызывает commit и close при выходе из контекста
    # conn.commit()

Типизация connection <class 'sqlite3.Connection'>, типизация cursor <class 'sqlite3.Cursor'>

Выполняем INSERT в базу...
Выполнили INSERT, закрываем соединение


После того, как таблицы созданы, можно читать из них данные

In [15]:
print("Открываем соединение с БД и читаем данные...\n")
with sqlite3.connect('my_database.db') as conn:
        c = conn.cursor()
        some_row = None
        for row in c.execute('SELECT * FROM jira_task LIMIT 10;'):
                print(row)
                some_row = row
        # conn.close()
        print("\nТипизация строки %s" % type(row))

Открываем соединение с БД и читаем данные...

('9,77311267424938', 'Treatment')
('9,31124557661141', 'Treatment')
('11,1728280618157', 'Treatment')
('11,6245100581039', 'Treatment')
('10,4623312287876', 'Treatment')
('10,3811435221337', 'Treatment')
('11,5479241097813', 'Treatment')
('11,6017357847944', 'Treatment')
('10,1714644311471', 'Treatment')
('12,1911555744297', 'Treatment')

Типизация строки <class 'tuple'>
